<div align=left style="width: 200; height: 80px; overflow: hidden">
    <img src=http://static1.squarespace.com/static/571446ff60b5e92c3a2b4249/57d8a40b9de4bb459f731cf3/58cb2f229de4bb4a049d38c2/1505340359463/teselaGenlogo.jpg align=right width=200>
</div>

# Hello World! (TEST module)

This notebook is intended to show how to use the Teselagen Python's client to access the TEST module. 

In [ ]:
import json
import platform
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from teselagen.api import TeselaGenClient
from teselagen.utils.utils import get_project_root

plt.style.use('seaborn-colorblind')

print(f"python version     : {platform.python_version()}")
print(f"json version       : {json.__version__}")
print(f"pandas version     : {pd.__version__}")
print(f"matplotlib version : {matplotlib.__version__}")

## Connect and Login

In [ ]:
# Connect to your teselagen instance by passing it as the 'host_url' argument of TESTCLient(host_url=host_url)
#host_url = "https://your.teselagen.instance.com"
#client = TESTClient(host_url = host_url)
client = TeselaGenClient()
# The following command will promt you to type username (email) and password
client.login()
# Alternatively you can add credential information in the arguments
#client.login(username="your@email", password="ZZZ")

## Laboratories

Laboratories are used to separate data access into groups of users. When you add an experiment into a lab it will only be seen by users that have access to that Lab. 

Now we list all available laboratories

In [ ]:
labs = client.get_laboratories()
display(labs)

At the next cell a lab is selected. The remaining actions of this notebook will be done inside that lab.

In [ ]:
## Select a Laboratory
client.select_laboratory(lab_name="Python")

## Experiments

Experiments represents all the studies inside a laboratory. Each experiment can contain several Assays. The **get_experiments** function returns a list of dictionaries that contain experiments information.

In [ ]:
## Get My Experiments from the selected Laboratory.
experiments = client.test.get_experiments()
# Print some of them
for exp in experiments: print(exp)

In [ ]:
## Create a new Experiment in the selected Laboratory.
new_experiment = client.test.create_experiment(experiment_name="Jupyter Experiment")
print(new_experiment)

## Assays

The assay object allows you to group data obtained from the same *protocol*.

Likewise other getters, the get_assays output can be easily parsed into a pandas dataframe. Here we show you how:

In [ ]:
## Get All Assays From all experiments in the selected Laboratory
assays_dataframe = pd.DataFrame(client.get_assays())
# Or, alternatevily, if you want to use id as index in the dataframe
#assays_dataframe = pd.DataFrame.from_records(client.get_assays(), index='id')
display(assays_dataframe)

You can also create assays using the client

In [ ]:
## Create a new Assay within a new experiment
new_assay = client.test.create_assay(experiment_id=new_experiment['id'], assay_name='Jupyter Assay')
print(new_assay)

And you can list all assays from an experiment

In [ ]:
## Get All Assays From Experiment with ID="experiment_id"
assays = pd.DataFrame(client.get_assays(experiment_id=new_experiment['id'])) 
display(assays)

## Files

The TEST module can also store files. Files are shared between users of the same lab and can be associated to specific assays.

In [ ]:
## Get All Files info from the selected Laboratory
pd.DataFrame(client.test.get_files_info()).head()
## You can fetch info from a specific assay using the assay_id argument
#display(pd.DataFrame(client.get_files(assay_id=new_assay['id'])))

In [ ]:
## Upload a File with an Assay.
filepath = str(get_project_root() / "teselagen/examples/test_module_example.csv")
new_file_info = client.test.upload_file(filepath=filepath, assay_id=new_assay['id'])
## Upload a File without an Assay.
#new_file_info = client.upload_file(filepath="./test_module_example.csv")
print(new_file_info)

In [ ]:
## Gets File data contens
binarydata = client.test.download_file(file_id=new_file_info['id'])
df_file = pd.read_csv(binarydata, sep=",")
df_file.head()

## Delete elements

You can delete an experiment to remove all its assays. Files work differently and you need to explicitly delete them. In the next cells we will download each object specifically:

In [ ]:
## Deletes a File
client.test.delete_file(file_id=new_file_info['id'])

In [ ]:
## Delete an Assay
client.test.delete_assay(assay_id=new_assay['id'])

In [ ]:
## Delete an Experiment from the selected laboratory.
client.test.delete_experiment(experiment_id=new_experiment['id'])

In [ ]:
# Delete all experiments with the name Jupyter Experiment 
# (if any remains due to uncompleted executions of the notebook)
# experiments = client.get_experiments()
# for exp in experiments:
#     if exp['name'] == "Jupyter Experiment":
#         client.delete_experiment(experiment_id=exp['id'])